## 네이버뉴스 언론사/일자별 동적 웹크롤링

- 실행 전 라이브러리 설치
- pip install webdriver-manager

- 코드 상단 📌 TARGET_URL 수정해서 그대로 실행
- 예시 URL : https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=009&date=20251218
- "oid=009&date=20251218"처럼 날짜 부분까지 나오도록 페이지 확인 후 입력 (언론사 들어가서 바로 나오는 오늘자 페이지는 날짜 표시가 바로 뜨지 않아서, 다른 일자 선택 후 다시 오늘 일자 선택)
- 👉 "oid=009&date=20251218.csv"로 본 파일과 같은 위치에 저장됨. (oid가 언론사 url 코드)

In [1]:
import time
import re
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ====================================================
# [크롤링 대상 URL] 📌 여기만 수정!
# ====================================================
TARGET_URL = "https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=008&date=20251218"

# ====================================================
# [보조 함수] 네이버 뉴스 날짜 텍스트를 표준 포맷으로 변환
# ====================================================
def clean_date(date_str):
    """
    입력: '2025.12.19. 오후 2:30' 또는 '기사입력 2025.12.19. 14:30' 등
    출력: '2025-12-19 14:30:00' (datetime 형식의 문자열)
    """
    try:
        # 1. 불필요한 글자 제거 (입력, 수정, 공백 등)
        # 정규식으로 숫자, 점(.), 오전/오후, 시간(:)만 남김
        date_str = date_str.replace("기사입력", "").replace("입력", "").strip()
        
        # 2. 오전/오후 처리 및 변환
        is_pm = "오후" in date_str
        date_str = date_str.replace("오전", "").replace("오후", "").strip()
        
        # 포맷 파싱 (YYYY.MM.DD. H:MM 또는 HH:MM)
        # 네이버는 보통 '2025.12.19. 2:30' 형태로 남음 (오전/오후 제거 후)
        dt = datetime.strptime(date_str, "%Y.%m.%d. %H:%M")
        
        # 오후이고 12시가 아니면 +12시간 / 오전이고 12시면 0시로 변경
        if is_pm and dt.hour != 12:
            dt = dt.replace(hour=dt.hour + 12)
        elif not is_pm and dt.hour == 12: # 오전 12시는 0시
            dt = dt.replace(hour=0)
            
        return dt.strftime("%Y-%m-%d %H:%M:%S")
        
    except Exception as e:
        # 변환 실패 시 (형식이 다르거나 날짜 없음) 원본 반환하되 에러 로그 출력하지 않음(조용히 처리)
        # 필요 시: print(f"날짜 변환 실패: {date_str} -> {e}")
        return date_str

# ====================================================
# 1. 크롬 드라이버 설정
# ====================================================
def set_chrome_driver():
    chrome_options = Options()
    # chrome_options.add_argument("--headless") 
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# ====================================================
# 2. 기사 상세 내용 추출 함수
# ====================================================
def extract_article_info(driver, url):
    try:
        driver.get(url)
        time.sleep(0.5)

        # 연예 기사 필터링
        if "entertain.naver.com" in driver.current_url:
            print(f" -> [패스] 연예 기사")
            return None

        # 제목 로딩 대기
        try:
            WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#title_area > span"))
            )
        except:
            if "entertain.naver.com" in driver.current_url:
                print(f" -> [패스] 연예 기사(로딩 후 확인)")
                return None
            return None

        # 1) 제목
        try:
            title = driver.find_element(By.CSS_SELECTOR, "#title_area > span").text
        except:
            title = "제목 없음"

        # 2) 본문 (캡션 제거)
        try:
            dic_area = driver.find_element(By.CSS_SELECTOR, "#dic_area")
            driver.execute_script("""
                var element = arguments[0];
                var dirts = element.querySelectorAll(".img_desc, .media_end_summary"); 
                for (var i = 0; i < dirts.length; i++) { dirts[i].remove(); }
            """, dic_area)
            content = dic_area.text.replace("\n", " ").strip()
        except:
            content = "본문 없음"

        # 3) 게재 시간 (추출 후 변환 적용)
        try:
            date_element = driver.find_element(By.CSS_SELECTOR, 
                "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span"
            )
            raw_date = date_element.text
        except:
            try:
                raw_date = driver.find_element(By.CSS_SELECTOR, ".media_end_head_info_datestamp span").text
            except:
                raw_date = "날짜 없음"
        
        # [수정됨] 날짜 표준화 함수 적용
        clean_date_str = clean_date(raw_date)

        print(f"[수집] {title} ({clean_date_str})")
        
        return {
            "날짜": clean_date_str, 
            "제목": title, 
            "본문": content, 
            "링크": url
        }

    except Exception as e:
        print(f"[에러] {e}")
        return None

# ====================================================
# 3. 메인 실행 함수
# ====================================================
def main():
    driver = set_chrome_driver()
    driver.get(TARGET_URL)
    time.sleep(2)

    collected_data = [] 
    page_count = 1      

    try:
        while True:
            print(f"\n===== [ {page_count} 페이지 ] =====")
            
            # --- URL 수집 ---
            article_urls = []
            selectors = [
                "#main_content > div.list_body.newsflash_body > ul.type06_headline > li dl > dt:not(.photo) > a",
                "#main_content > div.list_body.newsflash_body > ul.type06 > li dl > dt:not(.photo) > a"
            ]

            for sel in selectors:
                links = driver.find_elements(By.CSS_SELECTOR, sel)
                for link in links:
                    url = link.get_attribute("href")
                    if url: article_urls.append(url)
            
            print(f"-> 기사 {len(article_urls)}개 탐색 시작")

            # --- 상세 페이지 순회 ---
            list_window = driver.current_window_handle 
            
            for url in article_urls:
                if "entertain.naver.com" in url: continue

                driver.execute_script("window.open('');")
                driver.switch_to.window(driver.window_handles[-1])
                
                data = extract_article_info(driver, url)
                if data: collected_data.append(data)
                
                driver.close()
                driver.switch_to.window(list_window)
            
            # --- 페이징 ---
            try:
                paging_area = driver.find_element(By.CSS_SELECTOR, "#main_content > div.paging")
                current_page_num = int(paging_area.find_element(By.TAG_NAME, "strong").text)
                next_page_num = current_page_num + 1
                
                next_page_btn = None
                try:
                    next_page_btn = paging_area.find_element(By.XPATH, f".//a[text()='{next_page_num}']")
                except:
                    next_page_btn = None
                
                if next_page_btn:
                    next_page_btn.click()
                    page_count += 1
                    time.sleep(2)
                    continue

                try:
                    driver.find_element(By.CSS_SELECTOR, "a.next").click()
                    page_count += 1
                    time.sleep(2)
                    continue
                except:
                    print("-> 수집 종료 (마지막 페이지)")
                    break

            except:
                break
                
    except KeyboardInterrupt:
        print("\n[중단] 사용자 요청으로 중단합니다.")
    
    finally:
        driver.quit()
        
        # ====================================================
        # 4. 저장 (컬럼 순서 변경 및 파일명 생성)
        # ====================================================
        print("\n[저장] CSV 파일 생성 중...")
        
        if collected_data:
            df = pd.DataFrame(collected_data)
            
            # [수정됨] 컬럼 순서 강제 지정 (날짜 -> 제목 -> 본문 -> 링크)
            df = df[['날짜', '제목', '본문', '링크']]
            
            try:
                if "oid=" in TARGET_URL:
                    params = TARGET_URL.split('?')[1]
                    oid = [p for p in params.split('&') if 'oid=' in p][0]
                    date_val = [p for p in params.split('&') if 'date=' in p][0]
                    filename = f"{oid}&{date_val}.csv"
                else:
                    raise ValueError
            except:
                now = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"naver_news_result_{now}.csv"
            
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            
            print(f"✅ 저장 완료! 파일명: {filename}")
            print(f"📊 총 {len(df)}건 저장됨")
        else:
            print("❌ 수집된 데이터가 없습니다.")

if __name__ == "__main__":
    main()


===== [ 1 페이지 ] =====
-> 기사 20개 탐색 시작
[수집] 미국 11월 CPI, 전년 대비 2.7% 상승…전망치·전월치 모두 밑돌아 (2025-12-18 23:05:00)
[수집] 운동으로 뺐다던 입짧은햇님..."하루 약 3번" '주사이모' 의혹에 발칵 (2025-12-18 23:01:00)
[수집] 연말 음주 귀갓길 비극...택시서 내리다 '콰당', 뒤차에 치여 숨졌다[영상] (2025-12-18 22:32:00)
 -> [패스] 연예 기사
[수집] 英 기준금리 4%→3.75%, 0.25%p 인하…"추가 완화는 신중히" (2025-12-18 22:11:00)
 -> [패스] 연예 기사
[수집] 제주 여행 갔다가 날벼락…20대 게하 직원, 만취 투숙객 성폭행 (2025-12-18 21:36:00)
[수집] 허위·조작정보 근절법, 與 주도 법사위 통과...22일 본회의 상정 (2025-12-18 21:24:00)
[수집] 구윤철 "한국 시장 매력 높일 정책 병행…환율 방어에 국민연금 동원 안 해" (2025-12-18 21:20:00)
[수집] 미국 의회, 국방수권법서 '한국 조선사 투자 유치 검토' 조항 삭제 (2025-12-18 21:17:00)
[수집] 신안산선 공사 현장 사고로 1명 사망…고용부·국토부 조사 착수 (2025-12-18 21:04:00)
[수집] "의사인 줄"...'불법 시술 인정' 키, 연예대상 MC 하차→가요대전 불참 (2025-12-18 21:01:00)
[수집] 국방부 "핵잠 연료 협상, 2년 내 완료…전작권 2단계 검증, 내년 마무리" (2025-12-18 21:00:00)
[수집] 한온시스템, 유증 발행가액 주당 2830원 확정…재무구조 개선 박차 (2025-12-18 20:58:00)
[수집] "국산" 농협도 깜빡 속았다...대형마트서 팔린 중국산 표고버섯 915톤 (2025-12-18 20:40:00)
[수집] 검찰, UN 출신 최정원 스토킹 혐의 사건 보완수사 요구 (2025-12-18 20:33:00)


[수집] 고양시, 킨텍스 태양광 발전시설 내년 1월 운영 (2025-12-18 16:18:00)
[수집] 시흥시 청소년·청년 정책 '원트랙'…내년부터 통합재단 출범 (2025-12-18 16:16:00)
[수집] SKY 붙고도 "안 갈래"→의대로?...수시 최초합격자 2415명 등록 포기 (2025-12-18 16:15:00)
[수집] 진원생명과학, 흡인 기반 DNA 백신 접종 기술 연구 국제학술지 게재 (2025-12-18 16:14:00)
[수집] 에스티큐브, 1월 ASCO GI에서 대장암 1b상 바이오마커 분석 결과 발표 (2025-12-18 16:14:00)
[수집] 구리시, 국가유공자 처우개선 위한 수당 인상 (2025-12-18 16:11:00)
[수집] 협진, 사명 '앤로보틱스'로 변경…기계·전자 융합 로봇기업으로 전환 (2025-12-18 16:11:00)
[수집] 政, 내년 보건의료 R&D 2조4251억 투입…"국가대표기술 전폭 지원" (2025-12-18 16:09:00)
[수집] K바이오헬스 수출, 내년엔 300억달러 넘는다 (2025-12-18 16:08:00)
[수집] 전재수 "현금 2000만원, 시계 1개에 부산 미래를 팔아먹겠나" (2025-12-18 16:07:00)
[수집] 한미 대북정책 갈등 봉합될까?…정동영 "평화공존, 참모들이 역할 해야" (2025-12-18 16:06:00)
[수집] 현대카드, 소비자보호실태평가 2회 연속 '양호'… 금융사 중 유일 (2025-12-18 16:05:00)
[수집] 창원특례시 문화지구 도시재생 국토부 공모에 선정 (2025-12-18 16:01:00)
[수집] 조지호 파면에 경찰 "예상된 결과"…조직 안정화 개시 기대감 (2025-12-18 16:00:00)

===== [ 8 페이지 ] =====
-> 기사 20개 탐색 시작
[수집] 발행어음 전장 뛰어드는 하나·신한...내년초 경쟁 본격화 (2025-12-18 15:59:00)
[수집] 아산 스마트시티 무인순찰로봇 구축.. 팀그릿·호서대 

[수집] "R&D 참여 기업에 혜택 강화" 새 정부 전략기술 내년 상반기 나온다 (2025-12-18 14:09:00)
[수집] 셀트리온, 美 공장 인수 완료 '초읽기'…신제품 시너지로 성장 모멘텀↑ (2025-12-18 14:08:00)
[수집] 미국서 '가챠' 잘못 팔면 295억 벌금 폭탄…콘진원 보고서 봤더니 (2025-12-18 14:06:00)
[수집] 영광군, 성과와 혁신 담은 '군정 10대 우수시책' 선정 (2025-12-18 14:06:00)
[수집] 정청래 만난 정원오 성동구청장 "먼길 가기 위한 채비 단단히" (2025-12-18 14:05:00)

===== [ 14 페이지 ] =====
-> 기사 20개 탐색 시작
[수집] 위기의 트럼프…경제정책 지지율 역대 최저 "민주당이 낫다" (2025-12-18 14:04:00)
[수집] 조달청, 건설현장 고위험 작업 특별점검 (2025-12-18 14:02:00)
[수집] 한국카툰협회, 창립 20주년 'Cartoon 20th' 전시회 (2025-12-18 14:01:00)
[수집] 윤호중 행안장관, 초고층건물 화재 대비·제설 대응체계 점검 (2025-12-18 14:01:00)
[수집] "서울 아파트값 또 올랐어" 46주째 발동동...19년만에 최고상승률 전망 (2025-12-18 14:00:00)
[수집] "남친·여친 어디서 만나지?"…2030 절반 이상 '이곳'서 연애 시작한다 (2025-12-18 14:00:00)
[수집] "은행예금보다 좋네"…원금 안까먹고 수익률 높은 IMA 상품 나왔다 (2025-12-18 13:59:00)
[수집] "359만→490만원" 웃돈 붙었다...'두 번 접는' 삼성폰, 2분 완판되더니 (2025-12-18 13:59:00)
[수집] 서울고 동창회, 서울인상에 구본창 작가.김예동씨 (2025-12-18 13:57:00)
[수집] 김정관 산업장관 만난 中企업계 "대·중소 상생 생태계 마련해달라" (2025-12-18 13:55:00)
 -> [패스] 연예 기사
[

[수집] 벌써 세 번째 "카카오에 폭발물 설치"…이번엔 제주, 전직원 대피 (2025-12-18 10:49:00)
[수집] 與허영 "청년정책 이행해야…국회 청년특별위원회 구성하자" 제안 (2025-12-18 10:47:00)
[수집] 국민은행 'KB희망금융센터' 서울·인천 개점…취약계층 재기 지원 (2025-12-18 10:47:00)
[수집] 농협금융 '국민성장펀드 추진회의' 열어 (2025-12-18 10:46:00)
[수집] 구글, 최신 AI 모델 '제미나이 3 플래시' 공개…효율·속도 ↑ (2025-12-18 10:46:00)
[수집] 신한은행, 10년간 127조 기술금융 공급…중기·뿌리산업 성장 뒷받침 (2025-12-18 10:46:00)
[수집] [프로필]만프레드 하러 현대차·기아 R&D본부장 사장 (2025-12-18 10:44:00)
[수집] 민주당 '최다선' 조정식 "내년 후반기 국회의장 출마 뜻 있다" (2025-12-18 10:44:00)
[수집] 현대차그룹, R&D·제조 수장 사장 승진…"미래 경쟁력 강화" (2025-12-18 10:44:00)
[수집] 김병기 "한동훈-한덕수 공동정부구상, 국정 수습 아닌 내란 엄호" (2025-12-18 10:43:00)
[수집] 회계사회, '회계기본법 제정 논의' 등 2025년 10대 뉴스 선정 (2025-12-18 10:41:00)
[수집] 경과원, 스타트업·대기업·VC '삼각편대'…경기스타트업협의회 가동 (2025-12-18 10:41:00)
[수집] 국힘 박정하 "도태우 윤리위원장설…부정선거·윤어게인 절연 어려워" (2025-12-18 10:40:00)
[수집] 마곡 도시개발구역 '유보지·산업시설용지→복합용지'로 변경 (2025-12-18 10:40:00)
[수집] 광주경찰, 연말연시 음주운전 근절 위한 출근길 숙취운전 단속 (2025-12-18 10:38:00)
 -> [패스] 연예 기사

===== [ 21 페이지 ] =====
-> 기사 20개 탐색 시작
[수집] '노동시장 AI 인재' 

[수집] 미·러, 이번 주말 마이애미서 우크라 종전 협상 (2025-12-18 07:54:00)

===== [ 27 페이지 ] =====
-> 기사 20개 탐색 시작
[수집] "SBS, 본업인 TV·광고 부진 지속…목표가 ↓"-DB (2025-12-18 07:53:00)
[수집] 국토부, 국회 공청회 열고 '모듈러 건축 특별법' 제정 본격 추진 (2025-12-18 07:47:00)
 -> [패스] 연예 기사
 -> [패스] 연예 기사
[수집] 경제계, 'AI 3강' 도약 위한 '생태계 종합 지원책' 정부에 건의 (2025-12-18 07:39:00)
[수집] "내 남편 변호사, 너 콩밥 먹어야 해"…택시기사에 폭언한 만취녀 (2025-12-18 07:00:00)
[수집] "콜라 마실 땐 이 담배" KT&G 전략 통했다...우즈벡 한류 바람 타고 '불티' (2025-12-18 07:00:00)
 -> [패스] 연예 기사
[수집] 푸틴 "우크라 전쟁 목표 달성할 것…대화 안되면 군사수단 동원" (2025-12-18 06:37:00)
[수집] 오라클 데이터센터 난항에 AI 회의론 확산…나스닥 1.8%↓[뉴욕마감] (2025-12-18 06:18:00)
[수집] '주한미군 현 수준 유지' 美 의회 통과…일방감축 제동 (2025-12-18 06:13:00)
[수집] 샤이니 키, 박나래와 다른 반응?…"인정하고 사과"vs"배신감" (2025-12-18 06:11:00)
[수집] "곰팡이 3000마리 거주"...당근에 뜬 '핫한' 2800만원짜리 빌라 (2025-12-18 06:03:00)
[수집] '프리미엄 TV 철옹성' 노리는 삼성·LG…'마이크로 RGB'로 승부수 (2025-12-18 06:01:00)
[수집] '외산 무덤' 일본 건설시장 뚫은 K-건축 솔루션..."JV가 열쇠" (2025-12-18 06:00:00)
[수집] 입에 거품 문 채 쓰러진 학생들...'3명 사망' 강릉 펜션 참사 전말[뉴스속오늘] (2025-12-18 06:00:00)
[수집] 확정이